Step 4

In [1]:
import pandas as pd
import os

In [2]:
import neo4j

In [3]:
from neo4j import GraphDatabase

In [4]:
from graphdatascience import GraphDataScience

In [5]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

In [6]:
NEO4J_AUTH = ( "neo4j",  "assignment3")

In [7]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

In [8]:
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

Unable to connect to the Neo4j DBMS. Trying again...
Unable to connect to the Neo4j DBMS. Trying again...
Unable to connect to the Neo4j DBMS. Trying again...
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset:

Step 5

In [9]:
pii_query = '''
MATCH (c:Client)-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-> (n) <-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-(d:Client)
WHERE id(c) < id(d)
RETURN c.id, d.id, count(*) AS freq
ORDER BY freq DESC
'''

In [10]:
gds.run_cypher(pii_query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_EMAIL|HAS_PHONE|HAS_SSN' instead)} {position: line: 2, column: 43, offset: 43} for query: '\nMATCH (c:Client)-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-> (n) <-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-(d:Client)\nWHERE id(c) < id(d)\nRETURN c.id, d.id, count(*) AS freq\nORDER BY freq DESC\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a futu

,c.id,d.id,freq
0,4952527271473904,4816336012071985,3
1,4883445100935916,4708373581412325,3
2,4658150168863397,4100374538108184,3
3,4673951123644611,4795773320377768,3
4,4192214340630620,4912097363222923,3
...,...,...,...
754,4910140986334626,4114683318919154,1
755,4454780847105236,4210575070378533,1
756,4721862020593706,4210575070378533,1
757,4445521165797820,4210575070378533,1


Step 6

In [11]:
project_query = '''
CALL gds.graph.project(
    'clientClusters' ,
    { Client: {
        label: 'Client' }
    },
    { SHARED_PII: {
        type: 'SHARED_PII',
        orientation: 'UNDIRECTED',
        properties: {
        count: {
        property: 'count' }
            }
        }
    }
)
YIELD graphName, nodeCount, relationshipCount
'''

In [12]:
gds.run_cypher(project_query)

,graphName,nodeCount,relationshipCount
0,clientClusters,2433,1518


Step 7

In [13]:
streaming_query = '''
CALL gds.wcc.stream(
    'clientClusters',
    {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
    }
)
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId, 
componentId AS clusterId
'''

In [14]:
pandasDF = driver.execute_query(
    streaming_query,
    database_="neo4j",
    result_transformer_=  neo4j.Result.to_df
)

In [15]:
print(type(pandasDF))

<class 'pandas.core.frame.DataFrame'>


In [16]:
with driver.session() as session:
    result = session.run(streaming_query)
    print(result.single())

<Record clientId='4997933060327094' clusterId=0>


C:\Users\Joaquin\anaconda3\envs\myenv\lib\site-packages\neo4j\_sync\work\result.py:623: UserWarning: Expected a result with a single record, but found multiple.
  stacklevel=1,


Clean Up

In [17]:
gds.run_cypher('''CALL gds.graph.drop('nativeGraph', false)''')

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL gds.graph.drop('nativeGraph', false)"


,graphName,database,databaseLocation,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
